In [1]:
import torch
import torch.nn as nn
import pandas as pd
import ast
from typing import Optional
from dataclasses import dataclass
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from xformers.factory.model_factory import xFormer, xFormerConfig
from xformers.components.positional_embedding import (PositionEmbedding, PositionEmbeddingConfig, register_positional_embedding)
from transformers import AdamW, get_linear_schedule_with_warmup, Adafactor
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
Triton is not available, some optimizations will not be enabled.
This is just a warning: No module named 'triton'
Triton is not available, FusedMLP will not be enabled.
Either FairScale or torch distributed is not available, MixtureOfExperts will not be exposed. Please install them if you would like to use MoE


In [2]:
PAD_IDX = 128
BOS_IDX = 129
EOS_IDX = 130
PAD_VALUE = 0.0

NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
EMB_SIZE=64
MAX_LEN = 256
SRC_VOCAB_SIZE = 128+3 # 0-127 representing From C-1 to G9, 128 for PAD_IDX, 129 for BOS, 130 for EOS
TGT_VOCAB_SIZE = 128+3
NHEAD = 4
HIDDEN_LAYER_MULTIPLIER = 4
DROPOUT = 0.2

TRAIN_SPLIT = 0.95

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # A GPU with memory >=8GB is capable of training
LEARNING_RATE = 1e-3
NUM_EPOCHS = 7
BATCH_SIZE = 64
REPEATING_PENALTY = 32.0

LOAD_PRETRAINED = False
MODEL_SAVE_PATH = './Model/MidiGen.pth'

In [3]:
class MidiDataset(Dataset):
    def __init__(self, csv_path='./Data/MidiDataset.csv'):
        self.data = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.data) - 2
        # return 50000
    
    def __getitem__(self, idx):
        current_row = self.data.loc[idx]
        next_row = self.data.loc[idx+1]
        return (
            {
                'sentence': current_row.at['Sentence'],
                'time_since_last_note': current_row.at['TimeSinceLastNoteStart'],
                'duration': current_row.at['Duration'],
                'velocity': current_row.at['Velocity']
            },
            {
                'sentence': next_row.at['Sentence'],
                'time_since_last_note': next_row.at['TimeSinceLastNoteStart'],
                'duration': next_row.at['Duration'],
                'velocity': next_row.at['Velocity']
            }
        )


add_special_tokens = lambda x: torch.cat((torch.tensor([BOS_IDX]), torch.tensor(ast.literal_eval(x)), torch.tensor([EOS_IDX])))
add_padding_values = lambda x: torch.cat((torch.tensor([PAD_VALUE]), torch.tensor(ast.literal_eval(x)), torch.tensor([PAD_VALUE])))

def collate_fn(batch):
    sentences1 = [add_special_tokens(item[0]['sentence']) for item in batch]
    time_since_last_note1 = [add_padding_values(item[0]['time_since_last_note']) for item in batch]
    duration1 = [add_padding_values(item[0]['duration']) for item in batch]
    velocity1 = [add_padding_values(item[0]['velocity'])/100 for item in batch]
    sentences2 = [add_special_tokens(item[1]['sentence']) for item in batch]
    time_since_last_note2 = [add_padding_values(item[1]['time_since_last_note']) for item in batch]
    duration2 = [add_padding_values(item[1]['duration']) for item in batch]
    velocity2 = [add_padding_values(item[1]['velocity'])/100 for item in batch]

    # Pad them to the same length per batch
    sentences1 = pad_sequence(sentences1, batch_first=True, padding_value=PAD_IDX)
    time_since_last_note1 = pad_sequence(time_since_last_note1, batch_first=True, padding_value=PAD_VALUE).unsqueeze(-1)
    duration1 = pad_sequence(duration1, batch_first=True, padding_value=PAD_VALUE).unsqueeze(-1)
    velocity1 = pad_sequence(velocity1, batch_first=True, padding_value=PAD_VALUE).unsqueeze(-1)
    extra1 = torch.cat([time_since_last_note1, duration1, velocity1], dim=-1)

    sentences2 = pad_sequence(sentences2, batch_first=True, padding_value=PAD_IDX)
    time_since_last_note2 = pad_sequence(time_since_last_note2, batch_first=True, padding_value=PAD_VALUE).unsqueeze(-1)
    duration2 = pad_sequence(duration2, batch_first=True, padding_value=PAD_VALUE).unsqueeze(-1)
    velocity2 = pad_sequence(velocity2, batch_first=True, padding_value=PAD_VALUE).unsqueeze(-1)
    extra2 = torch.cat([time_since_last_note2, duration2, velocity2], dim=-1)

    return (sentences1, extra1), (sentences2, extra2)


In [4]:
dataset = MidiDataset()
train_split = int(len(dataset) * TRAIN_SPLIT)
train_dataset, val_dataset = random_split(dataset, [train_split, len(dataset) - train_split])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

NUM_TRAINING_STEPS = len(train_dataset) // BATCH_SIZE * NUM_EPOCHS
NUM_WARMUP_STEPS = NUM_TRAINING_STEPS // NUM_EPOCHS # Use 1 epoch for warmup

In [5]:
@dataclass
class MidiEmbeddingConfig(PositionEmbeddingConfig):
    pitch_size: int
    dropout: float


@register_positional_embedding("midi", MidiEmbeddingConfig)
class MidiEmbedding(PositionEmbedding):
    def __init__(self, dim_model: int, seq_len: int, pitch_size: int, dropout: float = 0.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dim_model = dim_model
        self.seq_len = seq_len
        self.pitch_size = pitch_size
        self.dropout = torch.nn.Dropout(p=dropout)

        self.position_embeddings = nn.Embedding(seq_len, self.dim_model)
        self.pitch_embeddings = nn.Embedding(
            self.pitch_size, self.dim_model - 3)

        self.position_ids: Optional[torch.Tensor] = None

    def init_weights(self, gain: float = 1.0):
        torch.nn.init.normal_(self.position_embeddings.weight, std=0.02 * gain)
        torch.nn.init.normal_(self.pitch_embeddings.weight, std=0.02 * gain)

    def forward(self, x: torch.Tensor):
        sentence = x[0]
        extra = x[1]

        position_ids = torch.arange(sentence.shape[1], dtype=torch.long, device=sentence.device)[
            None, :
        ].repeat(sentence.shape[0], 1)

        pitch_token = self.pitch_embeddings(sentence)

        x = torch.cat([pitch_token, extra], dim=-1)
        pos = self.position_embeddings(position_ids)

        X = x + pos
        X = self.dropout(X)

        return X


class MidiTransformer(nn.Module):
    def __init__(self, model_config) -> None:
        super().__init__()
        self.dim_model = model_config[1]['dim_model']
        self.model_config = xFormerConfig(model_config)
        self.transformer = xFormer.from_config(self.model_config)
        self.generator = nn.Sequential(
            nn.Linear(self.dim_model, self.dim_model*2), 
            nn.LeakyReLU(), 
            nn.Linear(self.dim_model*2, model_config[1]['position_encoding_config']['pitch_size']))
        self.extra_generator = nn.Sequential(
            nn.Linear(self.dim_model, self.dim_model*2), 
            nn.Linear(self.dim_model*2, 3))  # [time_since_last_note, duration, velocity]

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        memory = self.encode(src, src_mask)
        out = self.decode(tgt, memory, tgt_mask)
        return self.generator(out), self.extra_generator(out)

    def encode(self, src, src_mask=None):
        encoders = self.transformer.encoders
        memory = src[:]
        if isinstance(encoders, torch.nn.ModuleList):
            for encoder in encoders:
                memory = encoder(memory, input_mask=src_mask)
        else:
            if self.transformer.rev_enc_pose_encoding:
                memory = self.transformer.rev_enc_pose_encoding(src)

            # Reversible Encoder
            x = torch.cat([memory, memory], dim=-1)

            # Apply the optional input masking
            if src_mask is not None:
                if x.dim() - src_mask.dim() > 1:
                    src_mask.unsqueeze(0)
                x += src_mask.unsqueeze(-1)

            x = encoders(x)
            memory = torch.stack(x.chunk(2, dim=-1)).mean(dim=0)
        return memory

    def decode(self, tgt, memory, tgt_mask=None):
        for decoder in self.transformer.decoders:
            tgt = decoder(target=tgt, memory=memory, input_mask=tgt_mask)
        return tgt


model_config = [
    {
        "reversible": True,  # Reversible encoder can save a lot memory when training
        "block_type": "encoder",
        "num_layers": NUM_ENCODER_LAYERS,
        "dim_model": EMB_SIZE,
        "residual_norm_style": "pre",
        "position_encoding_config": {
            "name": "midi",  # The vocab type position encoding includes token embedding layer and position encoding layer
            "seq_len": MAX_LEN,
            "pitch_size": SRC_VOCAB_SIZE,
        },
        "multi_head_config": {
            "num_heads": NHEAD,
            "residual_dropout": 0,
            "attention": {
                "name": "linformer",
                "dropout": 0,
                "causal": False,
                "seq_len": MAX_LEN,
            },
        },
        "feedforward_config": {
            "name": "MLP",
            "dropout": DROPOUT,
            "activation": "relu",
            # Hidden layer dimension is HIDDEN_LAYER_MULTIPLIER times dim_model
            "hidden_layer_multiplier": HIDDEN_LAYER_MULTIPLIER,
        },
    },
    {
        "reversible": False,
        "block_type": "decoder",
        "num_layers": NUM_DECODER_LAYERS,
        "dim_model": EMB_SIZE,
        "residual_norm_style": "pre",
        "position_encoding_config": {
            "name": "midi",
            "seq_len": MAX_LEN,
            "pitch_size": TGT_VOCAB_SIZE,
        },
        "multi_head_config_masked": {
            "num_heads": NHEAD,
            "residual_dropout": 0,
            "attention": {
                "name": "nystrom",
                "dropout": 0,
                "causal": True,  # Causal attention is used to prevent the decoder from attending the future tokens in the target sequences
                "seq_len": MAX_LEN,
            },
        },
        "multi_head_config_cross": {
            "num_heads": NHEAD,
            "residual_dropout": 0,
            "attention": {
                "name": "favor",
                "dropout": 0,
                "causal": False,
                "seq_len": MAX_LEN,
            },
        },
        "feedforward_config": {
            "name": "MLP",
            "dropout": DROPOUT,
            "activation": "relu",
            "hidden_layer_multiplier": HIDDEN_LAYER_MULTIPLIER,
        },
    },
]


In [6]:
model = MidiTransformer(model_config=model_config)
print(f'Num Params: {sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6:.2f} M', model)
model = model.to(DEVICE)

cross_entropy = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
mae = nn.L1Loss()
# optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
optimizer = Adafactor(model.parameters(), lr=LEARNING_RATE, scale_parameter=False, relative_step=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=NUM_WARMUP_STEPS, num_training_steps=NUM_TRAINING_STEPS)

Num Params: 0.98 M MidiTransformer(
  (transformer): xFormer(
    (rev_enc_pose_encoding): MidiEmbedding(
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(256, 64)
      (pitch_embeddings): Embedding(131, 61)
    )
    (encoders): ReversibleSequence(
      (blocks): ModuleList(
        (0-5): 6 x ReversibleBlock(
          (f): Deterministic(
            (net): PreNorm(
              (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
              (sublayer): MultiHeadDispatch(
                (attention): LinformerAttention(
                  (E): Linear(in_features=256, out_features=64, bias=False)
                  (F): Linear(in_features=256, out_features=64, bias=False)
                  (attn_drop): Dropout(p=0, inplace=False)
                )
                (in_proj_container): InputProjection(
                  (q_proj): Linear(in_features=64, out_features=64, bias=True)
                  (k_proj): Linear(in_features=64, out_feat

In [7]:
if LOAD_PRETRAINED:
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [8]:
def accuracy(logits, targets, ignore_index=None):
    # Get the index of the highest logit value for each example
    predictions = torch.argmax(logits, dim=1)
    # Compare predictions to targets
    correct = (predictions == targets)
    # Ignore specified index
    if ignore_index is not None:
        mask = (targets != ignore_index)
        correct = correct[mask]
    # Calculate accuracy
    acc = correct.float().mean()
    return acc.item()

def repeating_penalty(logits, targets, ignore_index=PAD_IDX):
    """
    Calculates the penalty for repeating incorrect predictions.

    Args:
        logits (torch.Tensor): The model's output logits.
        targets (torch.Tensor): The target values.
        ignore_index (int, optional): Specifies a target value that is ignored when
            calculating the penalty. If `None`, no target value is ignored.

    Returns:
        torch.Tensor: The penalty for repeating incorrect predictions.
    """
    # Get the index of the highest logit value for each example
    predictions = torch.argmax(logits, dim=-1)
    # Compare predictions to targets
    incorrect = (predictions != targets.squeeze(-1))

    # If ignore_index is specified, ignore those targets
    if ignore_index is not None:
        ignored = (targets == ignore_index)
        incorrect[ignored] = False

    # Calculate penalty for repeating incorrect predictions
    penalty = (incorrect[:, 1:] & (predictions[:, 1:] == predictions[:, :-1])).float().sum()/(targets.shape[0] * targets.shape[1])
    return penalty


In [9]:
training_step = 0 # Used to trace global steps and plot in tensorboard
put_tuple_to_device = lambda x: tuple(x.to(DEVICE) for x in x)
create_mask = lambda x: x != PAD_IDX
writer = SummaryWriter()

def train_epoch(model, optimizer):
    model.train()
    for src, tgt in tqdm(train_dataloader, desc="Train"):
        src = put_tuple_to_device(src)
        tgt = put_tuple_to_device(tgt)
        tgt_input = (tgt[0][:, :-1], tgt[1][:, :-1])
        src_mask, tgt_mask = create_mask(src[0]), create_mask(tgt_input[0])

        optimizer.zero_grad()
        logits, extra = model(src, tgt_input, src_mask, tgt_mask)
        tgt_out = tgt[0][:, 1:]
        
        loss_c = cross_entropy(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss_h = mae(tgt[1][:, 1:].reshape(-1), extra.reshape(-1))
        loss_repeating = repeating_penalty(logits, tgt_out)
        loss = loss_c + loss_h + loss_repeating * REPEATING_PENALTY
        
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            acc = accuracy(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1), ignore_index=PAD_IDX)

        # Add tracing to tensorboard
        global training_step
        writer.add_scalar("Train/CrossEntropy", loss_c.item(), training_step)
        writer.add_scalar("Train/Accuracy", acc, training_step)
        writer.add_scalar("Train/MAE", loss_h.item(), training_step)
        writer.add_scalar("Train/Repeating", loss_repeating.item(), training_step)
        writer.add_scalar("Train/Total", loss.item(), training_step)
        writer.add_scalar("Learning Rate", optimizer.param_groups[0]["lr"], training_step)
        training_step += 1

def eval_epoch(model):
    model.eval()
    total_losses = 0
    total_losses_c = 0
    total_losses_h = 0
    total_losses_repeating = 0
    total_acc = 0
    total_steps = 0
    with torch.no_grad():
        for src, tgt in tqdm(val_dataloader, desc="Validation"):
            src = put_tuple_to_device(src)
            tgt = put_tuple_to_device(tgt)
            tgt_input = (tgt[0][:, :-1], tgt[1][:, :-1])
            src_mask, tgt_mask = create_mask(src[0]), create_mask(tgt_input[0])

            logits, extra = model(src, tgt_input, src_mask, tgt_mask)
            tgt_out = tgt[0][:, 1:]
            loss_c = cross_entropy(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            loss_h = mae(tgt[1][:, 1:].reshape(-1), extra.reshape(-1))
            loss_repeating = repeating_penalty(logits, tgt_out)
            loss = loss_c + loss_h + loss_repeating * REPEATING_PENALTY
            total_losses += loss.item()
            total_losses_c += loss_c.item()
            total_losses_h += loss_h.item()
            total_losses_repeating += loss_repeating.item()
            total_steps += 1
            total_acc += accuracy(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1), ignore_index=PAD_IDX)

        global training_step
        writer.add_scalar("Val/CrossEntropy", total_losses_c/total_steps, training_step)
        writer.add_scalar("Val/Accuracy", total_acc/total_steps, training_step)
        writer.add_scalar("Val/MAE", total_losses_h/total_steps, training_step)
        writer.add_scalar("Val/Repeating", total_losses_repeating/total_steps, training_step)
        writer.add_scalar("Val/Total", total_losses/total_steps, training_step)
    return total_losses / total_steps


In [10]:
from timeit import default_timer as timer
for epoch in range(NUM_EPOCHS):
    start_time = timer()
    print("-" * 64)
    print(f"Start Epoch {epoch + 1}/{NUM_EPOCHS}")
    train_epoch(model, optimizer)
    end_time = timer()
    eval_loss = eval_epoch(model)
    writer.flush()
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"End Epoch {epoch + 1}/{NUM_EPOCHS} in {end_time - start_time:.2f}s with eval loss {eval_loss:.2f}")
    print("-" * 64)

----------------------------------------------------------------
Start Epoch 1/7


Train:   0%|          | 0/5170 [00:00<?, ?it/s]

Validation:   0%|          | 0/273 [00:00<?, ?it/s]

End Epoch 1/7 in 975.01s with eval loss 8.24
----------------------------------------------------------------
----------------------------------------------------------------
Start Epoch 2/7


Train:   0%|          | 0/5170 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/273 [00:00<?, ?it/s]

End Epoch 6/7 in 978.92s with eval loss 3.73
----------------------------------------------------------------
----------------------------------------------------------------
Start Epoch 7/7


Train:   0%|          | 0/5170 [00:00<?, ?it/s]

Validation:   0%|          | 0/273 [00:00<?, ?it/s]

End Epoch 7/7 in 984.23s with eval loss 3.66
----------------------------------------------------------------


In [16]:
def greedy_decode(model, src, src_mask=None, max_len=MAX_LEN, start_symbol=BOS_IDX):
    src = put_tuple_to_device(src)
    memory = model.encode(src, src_mask)
    ys_token = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    ys_extra = torch.ones(1, 1, 3).fill_(PAD_VALUE).type(torch.long).to(DEVICE)
    ys = (ys_token, ys_extra)
    for i in range(max_len-1):
        out = model.decode(ys, memory)
        prob = model.generator(out[:, -1, :])
        extra = model.extra_generator(out[:, -1, :]).unsqueeze(0)
        _, next_token = torch.max(prob, dim=1)
        next_token = next_token.item()
        ys_token = torch.cat([ys_token, torch.ones(1, 1).type_as(src[0].data).fill_(next_token)], dim=1)
        ys_extra = torch.cat([ys_extra, extra], dim=1)
        ys = (ys_token, ys_extra)
        if next_token == EOS_IDX:
            break
    return ys


def predict_next_midi_sentence(model, src):
    model.eval()
    with torch.no_grad():
        mask = create_mask(src[0]).to(DEVICE)
        result = greedy_decode(model, src, mask)
    return result


src, trg = next(iter(val_dataloader))
src_test = (src[0][0].unsqueeze(0), src[1][0].unsqueeze(0))
print(f'Input Midi Data: {src_test[0]}')
midi_next = predict_next_midi_sentence(model, src_test)
print(f'Predicted Next Midi Sentence: {midi_next[0]}')

Input Midi Data: tensor([[129,  55,  57,  67,  69,  61,  63,  69,  67,  79,  81,  27,  39,  41,
          29,  43,  31,  49,  61,  63,  49,  45,  33,  49,  48,  36,  63,  61,
          49,  44,  32,  49,  47,  35,  63,  61, 130, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
         128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]])
Predicted Next Midi Sentence: tensor([[129,  49,  37,  49,  39,  51,  49,  37,  49,  39,  51,  49,  37,  49,
          39,  51,  49,  37,  49,  39,  51,  49,  37,  49,  39,  51,  49,  39,
          49,  39,  49,  51,  49, 130]], device='cuda:0')
